In [50]:
import itertools, json
from IPython.display import clear_output

def find_item_by_id(item_id, data):
    search = list(filter(lambda x: x['id'] == item_id, data))
    if(len(search) == 1):
        return search[0]
    elif(len(search) > 1):
        raise Exception("Not find.")
    
def get_id_users(data):
    return list(map(lambda x: x['id'], data))
    
def get_movies(user_id, data):
    user = find_item_by_id(user_id, data)
    return user['filmes']

def non_watch_movies(user_movies, neighborhood, data):
    all_nb_movies = list(map(lambda x: get_movies(x[0], data), neighborhood)) # all movies seen by the neighborhood
    all_nb_movies = list(itertools.chain.from_iterable(all_nb_movies)) # [[]] -> []
    all_nb_movies = list(set(map(lambda x: x['id'], all_nb_movies))) # IDs of all neighborhood movies
    u_movies = list(map(lambda x: x['id'], user_movies)) # IDs of user movies
    
    return list(filter(lambda x: x not in u_movies, all_nb_movies)) # movies not seen by the user in his neighborhood

def just_famous_movies(neighborhood, movies, n_movies, data): # (neighborhood) nearest neighborhood (e.g. 40 nearest users by similarity)
    nb_ids = list(map(lambda x: x[0], neighborhood)) # id of (id, similarity)
    nb = list(filter(lambda x: x['id'] in nb_ids, data)) # neighbors by id
    filmes_nb = list(map(lambda x: x['filmes'], nb)) # rating of neighbors [[]]
    filmes_nb = list(itertools.chain.from_iterable(filmes_nb)) # [[]] -> []
    
    if len(filmes_nb) == 0:
        return {'famous_movies': []}
    
    filmes_nb = list(map(lambda x: x['id'], filmes_nb)) # IDs of filmes    
    filmes_nb = list(map(lambda x: (x, filmes_nb.count(x)), movies)) # (movie, frequency)
    filmes_nb.sort(key=lambda x: x[1], reverse=True) # sort by highest frequency
    famous_movies = filmes_nb[:n_movies] # n popular movies
    famous_movies = list(map(lambda x: x[0], famous_movies)) # only IDs, frequency is not necessary, just for ranking
    
#     if len(famous_movies) > 0: # hard to happen, but...
        # search neighbors who watched at least one popular movie
    nb_watched_famous_movies = list(filter(lambda x: any(movie in famous_movies for movie in list(map(lambda y: y['id'], x['filmes'])) ), nb))
        
        # return in 'nb' the users obj to ease later, but it can be just IDs
    return {'famous_movies': famous_movies, 'nb': nb_watched_famous_movies}
#     else:
#         print("O valor de n = ", n, " é muito grande. Não existe um filme que ", n, " vizinhos assistiram.")
#         return {}
    
def find_similarity(neighborhood, user_id):
    search = list(filter(lambda x: x[0] == user_id, neighborhood))
    if(len(search) == 1):        
        return search[0][1] # return only similarity value
    else:
        raise Exception('not find')

all_predict = []
unpredictable = []
dataset_test = json.load(open('./test.json'))
def predict_user_movie(user, n, min_movies, data):
#     n = 2 # neighborhood
#     min_movies = 1 # min value of filmes
    
    neighborhood = user['user_similarity']
    neighborhood.sort(key=lambda x: x[1], reverse=True) # sort by similarity
    neighborhood = neighborhood[:n] # the real neighborhood
    
    # movies watched by neighborhood but not by the user
#     non_watched_movies = non_watch_movies(user['filmes'], neighborhood, data)

    # just the movies with high frequency within neighborhood
#     famous_movies = just_famous_movies(neighborhood, non_watched_movies, min_movies, data)
    
#     if famous_movies['famous_movies'] == []:
#         unpredictable.append({'user_id': user['id']})
#     else:
#         nb = famous_movies['nb'] # users
#         famous_movies = famous_movies['famous_movies'] # movies IDs

#         nb_ids = list(map(lambda x: x['id'], nb)) # just IDs
#         neighborhood = list(filter(lambda x: x[0] in nb_ids, neighborhood)) # filter by IDs -> (user_id, similarity)

    pa_is = [] # final prediction (movie_id, prediction_value)
    
    nb = list(map(lambda x: find_item_by_id(x[0],data), neighborhood))
    main_user = find_item_by_id(user['id'], dataset_test)
    famous_movies = list(map(lambda x: x['id'], main_user['filmes']))
    
    for movie_id in famous_movies:
        sum_1 = 0
        sum_2 = 0
        for i in nb:
            try:
                sim = find_similarity(neighborhood, i['id']) # search in list [(user_id, similarity)]
                mv = find_item_by_id(movie_id, i['filmes']) # retrieve movie in user movies list
                sum_1 = sum_1 + ((mv['avaliacao'] - i['media']) * sim)
                sum_2 = sum_2 + sim
            except:
#                 sum_1 = sum_1 + ((0 - i['media']) * sim)
                sum_2 = sum_2 + sim
        if sum_2 > 0:
            pa_i = user['media'] + (sum_1/sum_2)
        else:
            pa_i = user['media']
        pa_is.append({'filme':movie_id, 'predicao': pa_i})

    all_predict.append({'id': user['id'], 'predicoes': pa_is})

    
dataset = json.load(open('./usuarios.json'))
users = get_id_users(dataset)
tam = len(users)
for id_user in users:
    clear_output()
    print("Restam:", tam-1)
    tam = tam-1
    user = find_item_by_id(id_user, dataset)
    predict_user_movie(user, 20, 5, dataset)

all_predict[:5]

# with open('predictable.json', 'w') as outfile:
#     json.dump(all_predict, outfile)
    
# with open('unpredictable.json', 'w') as outfile:
#     json.dump(unpredictable, outfile)

Restam: 0


[{'id': 1488844,
  'predicoes': [{'filme': '17', 'predicao': 2.883454734651405}]},
 {'id': 30878, 'predicoes': [{'filme': '18', 'predicao': 2.8650933464891755}]},
 {'id': 823519, 'predicoes': [{'filme': '8', 'predicao': 2.466666666666667}]},
 {'id': 1181550,
  'predicoes': [{'filme': '16', 'predicao': 3.5203571428571427}]},
 {'id': 1227322,
  'predicoes': [{'filme': '8', 'predicao': 2.3877638144027324},
   {'filme': '17', 'predicao': 2.3960884762133463}]}]

# Separação dos dados disponíveis em treinamento e testes

In [1]:
from collections import Counter
import json, math
from random import randrange

dataset = json.load(open('./combined_data_1_1M.json'))

def reservoir_sampling(items, k):
    """ 
    Reservoir sampling algorithm for large sample space or unknow end list
    See <http://en.wikipedia.org/wiki/Reservoir_sampling> for detail>
    Type: ([a] * Int) -> [a]
    Prev constrain: k is positive and items at least of k items
    Post constrain: the length of return array is k
    Credits: https://gist.github.com/m00nlight/bfe54d1b2db362755a3a
    """
    sample = items[0:k]
    
    for i in range(k, len(items)):
        j = randrange(1, i + 1)
        if j < k:
            sample[j] = items[i]

    return sample

filtered = list(filter(lambda x: len(x['filmes']) >= 3, dataset))
# filtered = list(filter(lambda x: len(x['filmes']) >= 24, dataset)) # for tests

trainset = []
testset = []
for item in filtered:
    train = reservoir_sampling(item['filmes'], math.floor(len(item['filmes'])*.75))
    test = list(filter(lambda x: x not in train, item['filmes']))
    trainset.append({'id': item['id'], 'media': item['media'], 'user_similarity': [], 'filmes': train})
    testset.append({'id': item['id'], 'media': item['media'], 'user_similarity': [], 'filmes': test})

with open('train.json', 'w') as trainfile:
    json.dump(trainset, trainfile)
with open('test.json', 'w') as testfile:
    json.dump(testset, testfile)

### Definir vizinhança dos usuários

In [1]:
import json
with open('train.json') as f:
    usuarios = json.load(f)
print(len(usuarios))

80000


In [ ]:
import copy, json, math
    
print("start")
lambda_differ = 0.3

def inter(f1, f2):
    ids = set(map(lambda x: x['id'], f2)).intersection(set(map(lambda x: x['id'], f1)))
    return list(ids)

def calculo(user, num):
    global usuarios
    global file
    global lambda_differ
    
    na = 0
    user_similarity = []
    for i in usuarios:
        if user['id'] != i['id']:
            somatorio_1 = 0
            somatorio_2 = 0
            somatorio_3 = 0
            sumdiffer = 0
            m = 0
            ids = inter(user['filmes'], i['filmes'])
            if len(ids) > 0:
                f1 = list(filter(lambda x: x['id'] in ids, user['filmes']))
                f2 = list(filter(lambda x: x['id'] in ids, i['filmes']))
                for x,y in enumerate(f1):
                    s1 = (y["avaliacao"] - user["media"])
                    s2 = (f2[x]["avaliacao"] - i["media"])
                    somatorio_1 += s1*s2
                    somatorio_2 += math.pow(s1, 2)
                    somatorio_3 += math.pow(s2, 2)
                    m = m + 1
                    sumdiffer += math.pow(y["avaliacao"] - f2[x]["avaliacao"], 2)
                raiz = math.sqrt( somatorio_2 * somatorio_3)
                if (m):
                    sumdiffer = math.sqrt( sumdiffer/m)
                else:
                    sumdiffer = 0
                if (raiz): #verifica divizao por zero
                    similaridade = somatorio_1 / raiz
                else:
                    similaridade = 0
                if(m < 50):
                    similaridade = similaridade / (50-m)
                similaridade = similaridade *  math.pow(lambda_differ, sumdiffer)
                if(similaridade > 0):
                    user_similarity.append((i["id"], similaridade))
#                     i["user_similarity"].append((user["id"], similaridade))
            else:
                na += 1
            
    user_similarity.sort(key=lambda x: x[1], reverse=True)
    temp = copy.deepcopy(user)
    temp['user_similarity'] = user_similarity[:50]
    with open('vizinhanca_03.json', 'a') as outfile:
        outfile.write(json.dumps(temp)+",\n")
        outfile.close()
    print("\tcompleto:", (((num+1)*100)/80000),"%")

with open('vizinhanca_03.json', 'a') as outfile:
    outfile.write("[\n")
    outfile.close()
    
tam = len(usuarios)
for i, user in enumerate(usuarios):
    calculo(user, i)
    
with open('vizinhanca_03.json', 'a') as outfile:
    outfile.write("]")
    outfile.close()

start
	completo: 0.00125 %
	completo: 0.0025 %
	completo: 0.00375 %
	completo: 0.005 %
	completo: 0.00625 %
	completo: 0.0075 %
	completo: 0.00875 %
	completo: 0.01 %
	completo: 0.01125 %
	completo: 0.0125 %
	completo: 0.01375 %
	completo: 0.015 %
	completo: 0.01625 %
	completo: 0.0175 %
	completo: 0.01875 %
	completo: 0.02 %
	completo: 0.02125 %
	completo: 0.0225 %
	completo: 0.02375 %
	completo: 0.025 %
	completo: 0.02625 %
	completo: 0.0275 %
	completo: 0.02875 %
	completo: 0.03 %
	completo: 0.03125 %
	completo: 0.0325 %
	completo: 0.03375 %
	completo: 0.035 %
	completo: 0.03625 %
	completo: 0.0375 %
	completo: 0.03875 %
	completo: 0.04 %
	completo: 0.04125 %
	completo: 0.0425 %
	completo: 0.04375 %
	completo: 0.045 %
	completo: 0.04625 %
	completo: 0.0475 %
	completo: 0.04875 %
	completo: 0.05 %
	completo: 0.05125 %
	completo: 0.0525 %
	completo: 0.05375 %
	completo: 0.055 %
	completo: 0.05625 %
	completo: 0.0575 %
	completo: 0.05875 %
	completo: 0.06 %
	completo: 0.06125 %
	comple

	completo: 0.51 %
	completo: 0.51125 %
	completo: 0.5125 %
	completo: 0.51375 %
	completo: 0.515 %
	completo: 0.51625 %
	completo: 0.5175 %
	completo: 0.51875 %
	completo: 0.52 %
	completo: 0.52125 %
	completo: 0.5225 %
	completo: 0.52375 %
	completo: 0.525 %
	completo: 0.52625 %
	completo: 0.5275 %
	completo: 0.52875 %
	completo: 0.53 %
	completo: 0.53125 %
	completo: 0.5325 %
	completo: 0.53375 %
	completo: 0.535 %
	completo: 0.53625 %
	completo: 0.5375 %
	completo: 0.53875 %
	completo: 0.54 %
	completo: 0.54125 %
	completo: 0.5425 %
	completo: 0.54375 %
	completo: 0.545 %
	completo: 0.54625 %
	completo: 0.5475 %
	completo: 0.54875 %
	completo: 0.55 %
	completo: 0.55125 %
	completo: 0.5525 %
	completo: 0.55375 %
	completo: 0.555 %
	completo: 0.55625 %
	completo: 0.5575 %
	completo: 0.55875 %
	completo: 0.56 %
	completo: 0.56125 %
	completo: 0.5625 %
	completo: 0.56375 %
	completo: 0.565 %
	completo: 0.56625 %
	completo: 0.5675 %
	completo: 0.56875 %
	completo: 0.57 %
	completo: 0.571

	completo: 1.02 %
	completo: 1.02125 %
	completo: 1.0225 %
	completo: 1.02375 %
	completo: 1.025 %
	completo: 1.02625 %
	completo: 1.0275 %
	completo: 1.02875 %
	completo: 1.03 %
	completo: 1.03125 %
	completo: 1.0325 %
	completo: 1.03375 %
	completo: 1.035 %
	completo: 1.03625 %
	completo: 1.0375 %
	completo: 1.03875 %
	completo: 1.04 %
	completo: 1.04125 %
	completo: 1.0425 %
	completo: 1.04375 %
	completo: 1.045 %
	completo: 1.04625 %
	completo: 1.0475 %
	completo: 1.04875 %
	completo: 1.05 %
	completo: 1.05125 %
	completo: 1.0525 %
	completo: 1.05375 %
	completo: 1.055 %
	completo: 1.05625 %
	completo: 1.0575 %
	completo: 1.05875 %
	completo: 1.06 %
	completo: 1.06125 %
	completo: 1.0625 %
	completo: 1.06375 %
	completo: 1.065 %
	completo: 1.06625 %
	completo: 1.0675 %
	completo: 1.06875 %
	completo: 1.07 %
	completo: 1.07125 %
	completo: 1.0725 %
	completo: 1.07375 %
	completo: 1.075 %
	completo: 1.07625 %
	completo: 1.0775 %
	completo: 1.07875 %
	completo: 1.08 %
	completo: 1.081

	completo: 1.53 %
	completo: 1.53125 %
	completo: 1.5325 %
	completo: 1.53375 %
	completo: 1.535 %
	completo: 1.53625 %
	completo: 1.5375 %
	completo: 1.53875 %
	completo: 1.54 %
	completo: 1.54125 %
	completo: 1.5425 %
	completo: 1.54375 %
	completo: 1.545 %
	completo: 1.54625 %
	completo: 1.5475 %
	completo: 1.54875 %
	completo: 1.55 %
	completo: 1.55125 %
	completo: 1.5525 %
	completo: 1.55375 %
	completo: 1.555 %
	completo: 1.55625 %
	completo: 1.5575 %
	completo: 1.55875 %
	completo: 1.56 %
	completo: 1.56125 %
	completo: 1.5625 %
	completo: 1.56375 %
	completo: 1.565 %
	completo: 1.56625 %
	completo: 1.5675 %
	completo: 1.56875 %
	completo: 1.57 %
	completo: 1.57125 %
	completo: 1.5725 %
	completo: 1.57375 %
	completo: 1.575 %
	completo: 1.57625 %
	completo: 1.5775 %
	completo: 1.57875 %
	completo: 1.58 %
	completo: 1.58125 %
	completo: 1.5825 %
	completo: 1.58375 %
	completo: 1.585 %
	completo: 1.58625 %
	completo: 1.5875 %
	completo: 1.58875 %
	completo: 1.59 %
	completo: 1.591

	completo: 2.04 %
	completo: 2.04125 %
	completo: 2.0425 %
	completo: 2.04375 %
	completo: 2.045 %
	completo: 2.04625 %
	completo: 2.0475 %
	completo: 2.04875 %
	completo: 2.05 %
	completo: 2.05125 %
	completo: 2.0525 %
	completo: 2.05375 %
	completo: 2.055 %
	completo: 2.05625 %
	completo: 2.0575 %
	completo: 2.05875 %
	completo: 2.06 %
	completo: 2.06125 %
	completo: 2.0625 %
	completo: 2.06375 %
	completo: 2.065 %
	completo: 2.06625 %
	completo: 2.0675 %
	completo: 2.06875 %
	completo: 2.07 %
	completo: 2.07125 %
	completo: 2.0725 %
	completo: 2.07375 %
	completo: 2.075 %
	completo: 2.07625 %
	completo: 2.0775 %
	completo: 2.07875 %
	completo: 2.08 %
	completo: 2.08125 %
	completo: 2.0825 %
	completo: 2.08375 %
	completo: 2.085 %
	completo: 2.08625 %
	completo: 2.0875 %
	completo: 2.08875 %
	completo: 2.09 %
	completo: 2.09125 %
	completo: 2.0925 %
	completo: 2.09375 %
	completo: 2.095 %
	completo: 2.09625 %
	completo: 2.0975 %
	completo: 2.09875 %
	completo: 2.1 %
	completo: 2.1012

...

	completo: 3.06 %
	completo: 3.06125 %
	completo: 3.0625 %
	completo: 3.06375 %
	completo: 3.065 %
	completo: 3.06625 %
	completo: 3.0675 %
	completo: 3.06875 %
	completo: 3.07 %
	completo: 3.07125 %
	completo: 3.0725 %
	completo: 3.07375 %
	completo: 3.075 %
	completo: 3.07625 %
	completo: 3.0775 %
	completo: 3.07875 %
	completo: 3.08 %
	completo: 3.08125 %
	completo: 3.0825 %
	completo: 3.08375 %
	completo: 3.085 %
	completo: 3.08625 %
	completo: 3.0875 %
	completo: 3.08875 %
	completo: 3.09 %
	completo: 3.09125 %
	completo: 3.0925 %
	completo: 3.09375 %
	completo: 3.095 %
	completo: 3.09625 %
	completo: 3.0975 %
	completo: 3.09875 %
	completo: 3.1 %
	completo: 3.10125 %
	completo: 3.1025 %
	completo: 3.10375 %
	completo: 3.105 %
	completo: 3.10625 %
	completo: 3.1075 %
	completo: 3.10875 %
	completo: 3.11 %
	completo: 3.11125 %
	completo: 3.1125 %
	completo: 3.11375 %
	completo: 3.115 %
	completo: 3.11625 %
	completo: 3.1175 %
	completo: 3.11875 %
	completo: 3.12 %
	completo: 3.1212

	completo: 3.57 %
	completo: 3.57125 %
	completo: 3.5725 %
	completo: 3.57375 %
	completo: 3.575 %
	completo: 3.57625 %
	completo: 3.5775 %
	completo: 3.57875 %
	completo: 3.58 %
	completo: 3.58125 %
	completo: 3.5825 %
	completo: 3.58375 %
	completo: 3.585 %
	completo: 3.58625 %
	completo: 3.5875 %
	completo: 3.58875 %
	completo: 3.59 %
	completo: 3.59125 %
	completo: 3.5925 %
	completo: 3.59375 %
	completo: 3.595 %
	completo: 3.59625 %
	completo: 3.5975 %
	completo: 3.59875 %
	completo: 3.6 %
	completo: 3.60125 %
	completo: 3.6025 %
	completo: 3.60375 %
	completo: 3.605 %
	completo: 3.60625 %
	completo: 3.6075 %
	completo: 3.60875 %
	completo: 3.61 %
	completo: 3.61125 %
	completo: 3.6125 %
	completo: 3.61375 %
	completo: 3.615 %
	completo: 3.61625 %
	completo: 3.6175 %
	completo: 3.61875 %
	completo: 3.62 %
	completo: 3.62125 %
	completo: 3.6225 %
	completo: 3.62375 %
	completo: 3.625 %
	completo: 3.62625 %
	completo: 3.6275 %
	completo: 3.62875 %
	completo: 3.63 %
	completo: 3.6312

	completo: 4.08 %
	completo: 4.08125 %
	completo: 4.0825 %
	completo: 4.08375 %
	completo: 4.085 %
	completo: 4.08625 %
	completo: 4.0875 %
	completo: 4.08875 %
	completo: 4.09 %
	completo: 4.09125 %
	completo: 4.0925 %
	completo: 4.09375 %
	completo: 4.095 %
	completo: 4.09625 %
	completo: 4.0975 %
	completo: 4.09875 %
	completo: 4.1 %
	completo: 4.10125 %
	completo: 4.1025 %
	completo: 4.10375 %
	completo: 4.105 %
	completo: 4.10625 %
	completo: 4.1075 %
	completo: 4.10875 %
	completo: 4.11 %
	completo: 4.11125 %
	completo: 4.1125 %
	completo: 4.11375 %
	completo: 4.115 %
	completo: 4.11625 %
	completo: 4.1175 %
	completo: 4.11875 %
	completo: 4.12 %
	completo: 4.12125 %
	completo: 4.1225 %
	completo: 4.12375 %
	completo: 4.125 %
	completo: 4.12625 %
	completo: 4.1275 %
	completo: 4.12875 %
	completo: 4.13 %
	completo: 4.13125 %
	completo: 4.1325 %
	completo: 4.13375 %
	completo: 4.135 %
	completo: 4.13625 %
	completo: 4.1375 %
	completo: 4.13875 %
	completo: 4.14 %
	completo: 4.1412

	completo: 4.59 %
	completo: 4.59125 %
	completo: 4.5925 %
	completo: 4.59375 %
	completo: 4.595 %
	completo: 4.59625 %
	completo: 4.5975 %
	completo: 4.59875 %
	completo: 4.6 %
	completo: 4.60125 %
	completo: 4.6025 %
	completo: 4.60375 %
	completo: 4.605 %
	completo: 4.60625 %
	completo: 4.6075 %
	completo: 4.60875 %
	completo: 4.61 %
	completo: 4.61125 %
	completo: 4.6125 %
	completo: 4.61375 %
	completo: 4.615 %
	completo: 4.61625 %
	completo: 4.6175 %
	completo: 4.61875 %
	completo: 4.62 %
	completo: 4.62125 %
	completo: 4.6225 %
	completo: 4.62375 %
	completo: 4.625 %
	completo: 4.62625 %
	completo: 4.6275 %
	completo: 4.62875 %
	completo: 4.63 %
	completo: 4.63125 %
	completo: 4.6325 %
	completo: 4.63375 %
	completo: 4.635 %
	completo: 4.63625 %
	completo: 4.6375 %
	completo: 4.63875 %
	completo: 4.64 %
	completo: 4.64125 %
	completo: 4.6425 %
	completo: 4.64375 %
	completo: 4.645 %
	completo: 4.64625 %
	completo: 4.6475 %
	completo: 4.64875 %
	completo: 4.65 %
	completo: 4.6512

	completo: 5.1 %
	completo: 5.10125 %
	completo: 5.1025 %
	completo: 5.10375 %
	completo: 5.105 %
	completo: 5.10625 %
	completo: 5.1075 %
	completo: 5.10875 %
	completo: 5.11 %
	completo: 5.11125 %
	completo: 5.1125 %
	completo: 5.11375 %
	completo: 5.115 %
	completo: 5.11625 %
	completo: 5.1175 %
	completo: 5.11875 %
	completo: 5.12 %
	completo: 5.12125 %
	completo: 5.1225 %
	completo: 5.12375 %
	completo: 5.125 %
	completo: 5.12625 %
	completo: 5.1275 %
	completo: 5.12875 %
	completo: 5.13 %
	completo: 5.13125 %
	completo: 5.1325 %
	completo: 5.13375 %
	completo: 5.135 %
	completo: 5.13625 %
	completo: 5.1375 %
	completo: 5.13875 %
	completo: 5.14 %
	completo: 5.14125 %
	completo: 5.1425 %
	completo: 5.14375 %
	completo: 5.145 %
	completo: 5.14625 %
	completo: 5.1475 %
	completo: 5.14875 %
	completo: 5.15 %
	completo: 5.15125 %
	completo: 5.1525 %
	completo: 5.15375 %
	completo: 5.155 %
	completo: 5.15625 %
	completo: 5.1575 %
	completo: 5.15875 %
	completo: 5.16 %
	completo: 5.1612